In [ ]:
! pip install telethon python-dotenv pandas sqlalchemy fastapi uvicorn torch torchvision dbt opencv-python

In [ ]:
!pip install nest_asyncio

In [3]:
import os
import csv
import logging
from telethon import TelegramClient
from dotenv import load_dotenv
import nest_asyncio
import asyncio

In [4]:
import os
from dotenv import load_dotenv
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [6]:
# Load API credentials from the .env file in Google Drive
load_dotenv('/content/drive/My Drive/Week 7 KAIM/.env')

True

In [7]:
# Retrieve API credentials and handle errors if not found
api_id = os.getenv('TG_API_ID')
api_hash = os.getenv('TG_API_HASH')
phone = os.getenv('phone')

if not api_id or not api_hash or not phone:
    raise ValueError("One or more environment variables are missing. Check your .env file.")

api_id = int(api_id)  # Convert api_id to integer

In [8]:
# Step 4: Use an in-memory session instead of a file-based session
client = TelegramClient('anon', api_id, api_hash)

In [9]:
# Step 5: Setup logging
logging.basicConfig(filename='scraping/scraping.log', level=logging.INFO)

In [10]:
# Step 6: Define the function to scrape messages from Telegram channels
async def scrape_channel(channel, writer, media_dir='scraping/photos'):
    os.makedirs(media_dir, exist_ok=True)
    entity = await client.get_entity(channel)

    async for message in client.iter_messages(entity, limit=50):
        media_path = None
        if message.media:
            filename = f"{channel}_{message.id}.jpg"
            media_path = os.path.join(media_dir, filename)
            await client.download_media(message.media, media_path)
        writer.writerow([channel, message.id, message.message, message.date, media_path])

In [11]:
# Step 7: Main async function to orchestrate scraping
async def main():
    await client.start(phone)
    with open('scraped_data.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Channel', 'Message ID', 'Content', 'Date', 'Media Path'])
        channels = ['@DoctorsEthiopia', 'https://t.me/EAHCI', 'https://t.me/lobelia4cosmetics']
        for channel in channels:
            await scrape_channel(channel, writer)
    await client.disconnect()

In [12]:
# Step 8: Apply nest_asyncio to allow running asyncio in an existing event loop
nest_asyncio.apply()

In [ ]:
# Step 9: Run the main function
asyncio.run(main())